# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. Using an RNN rather than a feedfoward network is more accurate since we can include information about the *sequence* of words. Here we'll use a dataset of movie reviews, accompanied by labels.

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=400px>

Here, we'll pass in words to an embedding layer. We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the word2vec lesson. You can actually train up an embedding with word2vec and use it here. But it's good enough to just have an embedding layer and let the network learn the embedding table on it's own.

From the embedding layer, the new representations will be passed to LSTM cells. These will add recurrent connections to the network so we can include information about the sequence of words in the data. Finally, the LSTM cells will go to a sigmoid output layer here. We're using the sigmoid because we're trying to predict if this text has positive or negative sentiment. The output layer will just be a single unit then, with a sigmoid activation function.

We don't care about the sigmoid outputs except for the very last one, we can ignore the rest. We'll calculate the cost from the output of the last step and the training label.

In [68]:
import numpy as np
import tensorflow as tf
from collections import Counter

In [69]:
#opening the files

with open ('../sentiment-network/reviews.txt', 'r') as r: #this finds the file in whatever directory
    reviews = list(map(lambda x : x[:-1], r.readlines()))
    #reviews = r.read()
    
with open ('../sentiment-network/labels.txt', 'r') as lab:
    labels = list(map(lambda x : x[:-1].upper(), lab.readlines()))

In [70]:

# rad = Counter([len(reviews[i]) for i in range(len(reviews))])

# sorted(rad.items(), reverse=True)

# len(reviews[3])



4476

## Data preprocessing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. We'll want to get rid of those periods. Also, you might notice that the reviews are delimited with newlines `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [71]:
#remove punctuations 
#append reviews with removed punctuations

all_list = []

from string import punctuation
for review in range(len(reviews)):
    worded = []
    for word in reviews[review]:
        if word not in punctuation:
            worded.append(word)
    word_join = ''.join(worded)
    all_list.append(word_join)
        
    
    


In [72]:
all_list[3]

'airport    starts as a brand new luxury    plane is loaded up with valuable paintings  such belonging to rich businessman philip stevens  james stewart  who is flying them  a bunch of vip  s to his estate in preparation of it being opened to the public as a museum  also on board is stevens daughter julie  kathleen quinlan   her son  the luxury jetliner takes off as planned but mid  air the plane is hi  jacked by the co  pilot chambers  robert foxworth   his two accomplice  s banker  monte markham   wilson  michael pataki  who knock the passengers  crew out with sleeping gas  they plan to steal the valuable cargo  land on a disused plane strip on an isolated island but while making his descent chambers almost hits an oil rig in the ocean  loses control of the plane sending it crashing into the sea where it sinks to the bottom right bang in the middle of the bermuda triangle  with air in short supply  water leaking in  having flown over    miles off course the problems mount for the sur

In [73]:
#Taking all the words in the vocabularly
word_list = []
for review in range(len(all_list)):
    for words in (all_list[review]).split (' '):
        word_list.append(words)
    


In [107]:
#removing the ''
word_listed = [i for i in word_list if i != '' ]

In [118]:
#lenght of words in reviews
len(set(word_listed))


74072

In [109]:
#counting number of words
word_counter = Counter(word_listed)
sorted_vocab = sorted(word_counter, key=word_counter.get, reverse=True)

In [98]:
word_counter[" "]

0

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [111]:
# Create your dictionary that maps vocab words to integers here
vocab_to_int = {word : numb  for numb, word in enumerate(sorted_vocab, 1)}

# Convert the reviews to integers, same shape as reviews list, but with integers
reviews_ints = []

for review in all_list:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [112]:
reviews_ints[2]

[22382,
 42,
 46418,
 15,
 706,
 17139,
 3389,
 47,
 77,
 35,
 1819,
 16,
 154,
 19,
 114,
 3,
 1305,
 5,
 336,
 147,
 22,
 1,
 857,
 12,
 70,
 281,
 1168,
 399,
 36,
 120,
 283,
 38,
 169,
 5,
 382,
 158,
 42,
 2269,
 16,
 1,
 541,
 90,
 78,
 102,
 4,
 1,
 3244,
 15,
 43,
 3,
 407,
 1068,
 136,
 8055,
 44,
 182,
 140,
 15,
 3043,
 1,
 320,
 22,
 4818,
 26224,
 346,
 5,
 3090,
 2092,
 1,
 18839,
 17939,
 42,
 8055,
 46,
 33,
 236,
 29,
 370,
 5,
 130,
 56,
 22,
 1,
 1928,
 7,
 7,
 19,
 48,
 46,
 21,
 70,
 344,
 3,
 2099,
 5,
 408,
 22,
 1,
 1928,
 16,
 3,
 3119,
 205,
 1,
 28787,
 21,
 281,
 68,
 38,
 3,
 339,
 1,
 700,
 715,
 3,
 3818,
 1229,
 22,
 1,
 1491,
 3,
 1197,
 2,
 283,
 21,
 281,
 2435,
 5,
 66,
 48,
 8,
 13,
 39,
 5,
 29,
 3244,
 12,
 6,
 21026,
 11723,
 13,
 2015,
 7,
 7,
 3687,
 2818,
 36,
 4147,
 36,
 374,
 15,
 11723,
 296,
 3,
 996,
 125,
 36,
 47,
 283,
 9,
 1,
 176,
 363,
 6893,
 5,
 94,
 3,
 2099,
 17,
 3,
 4976,
 2932,
 14557,
 19870,
 5,
 66,
 46,
 25,
 51,
 408,


In [113]:
vocab_to_int

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'it': 8,
 'in': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 's': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 'you': 21,
 'on': 22,
 't': 23,
 'not': 24,
 'he': 25,
 'are': 26,
 'his': 27,
 'have': 28,
 'be': 29,
 'one': 30,
 'all': 31,
 'at': 32,
 'they': 33,
 'by': 34,
 'an': 35,
 'who': 36,
 'so': 37,
 'from': 38,
 'like': 39,
 'there': 40,
 'her': 41,
 'or': 42,
 'just': 43,
 'about': 44,
 'out': 45,
 'if': 46,
 'has': 47,
 'what': 48,
 'some': 49,
 'good': 50,
 'can': 51,
 'more': 52,
 'she': 53,
 'when': 54,
 'very': 55,
 'up': 56,
 'time': 57,
 'no': 58,
 'even': 59,
 'my': 60,
 'would': 61,
 'which': 62,
 'story': 63,
 'only': 64,
 'really': 65,
 'see': 66,
 'their': 67,
 'had': 68,
 'we': 69,
 'were': 70,
 'me': 71,
 'well': 72,
 'than': 73,
 'much': 74,
 'get': 75,
 'bad': 76,
 'been': 77,
 'people': 78,
 'will': 79,
 'do': 80,
 'other': 81,
 'also': 82,
 'into':

### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively.

In [119]:
# Convert labels to 1s and 0s for 'positive' and 'negative
label_int = []
for label in labels:
    if (label == 'POSITIVE'):
        label_int.append(1)
    else:
        label_int.append(0)

In [81]:
label_int[1]

0

If you built `labels` correctly, you should see the next output.

In [120]:
from collections import Counter
review_lens = Counter([len(reviews_ints[x]) for x in range(len(reviews_ints))])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 2514


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. Let's truncate to 200 steps. For reviews shorter than 200, we'll pad with 0s. For reviews longer than 200, we can truncate them to the first 200 characters.

> **Exercise:** First, remove the review with zero length from the `reviews_ints` list.

> **Exercise:** Now, create an array `features` that contains the data we'll pass to the network. The data should come from `review_ints`, since we want to feed integers to the network. Each row should be 200 elements long. For reviews shorter than 200 words, left pad with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. For reviews longer than 200, use on the first 200 words as the feature vector.

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.



In [122]:
def zero_list_maker(n):
    return [0] * n

In [123]:
#implementing padded reviews of specific sequence length

seq_len = 200

features = []

for review in range(len(reviews_ints)):
    standard = zero_list_maker(seq_len)
    count = len(reviews_ints[review])
    if (count < 200):
        standard[-count :] = reviews_ints[review]
                    
    elif (count > 200):
        standard = reviews_ints[review][:seq_len]
                    
    features.append(standard)
        
                    


If you build features correctly, it should look like that cell output below.

In [124]:
features = np.array(features)
labels =  np.array(label_int)

In [126]:
features[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0, 21025,   308,     6,
           3,  1050,   207,     8,  2138,    32,     1,   171,    57,
          15,    49,    81,  5785,    44,   382,   110,   140,    15,
        5194,    60,   154,     9,     1,  4975,  5852,   475,    71,
           5,   260,    12, 21025,   308,    13,  1978,     6,    74,
        2395,     5,   613,    73,     6,  5194,     1, 24103,     5,
        1983, 10166,     1,  5786,  1499,    36,    51,    66,   204,
         145,    67,  1199,  5194, 19869,     1, 37442,     4,     1,
         221,   883,

## Training, Validation, Test



With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets here. You'll need to create sets for the features and the labels, `train_x` and `train_y` for example. Define a split fraction, `split_frac` as the fraction of data to keep in the training set. Usually this is set to 0.8 or 0.9. The rest of the data will be split in half to create the validation and testing data.

In [146]:
#creating train, validation and test sets

split_frac = 0.8


train_x, val_x, = features[: int(0.8 * cout)], features[int(0.8 * cout) :]
train_y, val_y = labels[: int(0.8 * cout)], labels[int(0.8 * cout) :]

val_x, test_x = val_x[: int(0.5 * len(val_x))], val_x[int(0.5 * len(val_x)) :]
val_y, test_y = val_y[: int(0.5 * len(val_y))], val_y[int(0.5 * len(val_y)) :]


print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


With train, validation, and text fractions of 0.8, 0.1, 0.1, the final shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

## Build the graph

Here, we'll build the graph. First up, defining the hyperparameters.

* `lstm_size`: Number of units in the hidden layers in the LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `lstm_layers`: Number of LSTM layers in the network. I'd start with 1, then add more if I'm underfitting.
* `batch_size`: The number of reviews to feed the network in one training pass. Typically this should be set as high as you can go without running out of memory.
* `learning_rate`: Learning rate

For the network itself, we'll be passing in our 200 element long review vectors. Each batch will be `batch_size` vectors. We'll also be using dropout on the LSTM layer, so we'll make a placeholder for the keep probability.

> **Exercise:** Create the `inputs_`, `labels_`, and drop out `keep_prob` placeholders using `tf.placeholder`. `labels_` needs to be two-dimensional to work with some functions later.  Since `keep_prob` is a scalar (a 0-dimensional tensor), you shouldn't provide a size to `tf.placeholder`.

In [157]:
n_words = len(vocab_to_int) + 1 # Adding 1 because we use 0's for padding, dictionary started at 1

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None])
    labels_ = tf.placeholder(tf.int32, [None, None])
    keep_prob = tf.placeholder(tf.float32)

### Embedding

Now we'll add an embedding layer. We need to do this because there are 74000 words in our vocabulary. It is massively inefficient to one-hot encode our classes here. You should remember dealing with this problem from the word2vec lesson. Instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using word2vec, then load it here. But, it's fine to just make a new layer and let the network learn the weights.

> **Exercise:** Create the embedding lookup matrix as a `tf.Variable`. Use that embedding matrix to get the embedded vectors to pass to the LSTM cell with [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup). This function takes the embedding matrix and an input tensor, such as the review vectors. Then, it'll return another tensor with the embedded vectors. So, if the embedding layer has 200 units, the function will return a tensor with size [batch_size, 200].



In [158]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    #with tf.variable_scope ('emdo'):
        embedding = tf.Variable(tf.random_uniform([n_words, embed_size], minval = -1, maxval = 1))
        embed = tf.nn.embedding_lookup(embedding, inputs_)

### LSTM cell

<img src="assets/network_diagram.png" width=400px>

Next, we'll create our LSTM cells to use in the recurrent network ([TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn)). Here we are just defining what the cells look like. This isn't actually building the graph, just defining the type of cells we want in our graph.

To create a basic LSTM cell for the graph, you'll want to use `tf.contrib.rnn.BasicLSTMCell`. Looking at the function documentation:

```
tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0, input_size=None, state_is_tuple=True, activation=<function tanh at 0x109f1ef28>)
```

you can see it takes a parameter called `num_units`, the number of units in the cell, called `lstm_size` in this code. So then, you can write something like 

```
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

to create an LSTM cell with `num_units`. Next, you can add dropout to the cell with `tf.contrib.rnn.DropoutWrapper`. This just wraps the cell in another cell, but with dropout added to the inputs and/or outputs. It's a really convenient way to make your network better with almost no effort! So you'd do something like

```
drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
```

Most of the time, your network will have better performance with more layers. That's sort of the magic of deep learning, adding more layers allows the network to learn really complex relationships. Again, there is a simple way to create multiple layers of LSTM cells with `tf.contrib.rnn.MultiRNNCell`:

```
cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
```

Here, `[drop] * lstm_layers` creates a list of cells (`drop`) that is `lstm_layers` long. The `MultiRNNCell` wrapper builds this into multiple layers of RNN cells, one for each cell in the list.

So the final cell you're using in the network is actually multiple (or just one) LSTM cells with dropout. But it all works the same from an architectural viewpoint, just a more complicated graph in the cell.

> **Exercise:** Below, use `tf.contrib.rnn.BasicLSTMCell` to create an LSTM cell. Then, add drop out to it with `tf.contrib.rnn.DropoutWrapper`. Finally, create multiple LSTM layers with `tf.contrib.rnn.MultiRNNCell`.

Here is [a tutorial on building RNNs](https://www.tensorflow.org/tutorials/recurrent) that will help you out.


In [159]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

### RNN forward pass

<img src="assets/network_diagram.png" width=400px>

Now we need to actually run the data through the RNN nodes. You can use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) to do this. You'd pass in the RNN cell you created (our multiple layered LSTM `cell` for instance), and the inputs to the network.

```
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
```

Above I created an initial state, `initial_state`, to pass to the RNN. This is the cell state that is passed between the hidden layers in successive time steps. `tf.nn.dynamic_rnn` takes care of most of the work for us. We pass in our cell and the input to the cell, then it does the unrolling and everything else for us. It returns outputs for each time step and the final_state of the hidden layer.

> **Exercise:** Use `tf.nn.dynamic_rnn` to add the forward pass through the RNN. Remember that we're actually passing in vectors from the embedding layer, `embed`.



In [160]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state = initial_state)

### Output

We only care about the final output, we'll be using that as our sentiment prediction. So we need to grab the last output with `outputs[:, -1]`, the calculate the cost from that and `labels_`.

In [168]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn = tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Validation accuracy

Here we can add a few nodes to calculate the accuracy which we'll use in the validation pass.

In [169]:
with graph.as_default():
    #labels_ = tf.cast(labels_, tf.int32)
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [143]:
len(train_x)

20000

### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the `x` and `y` arrays and returns slices out of those arrays with size `[batch_size]`.

In [189]:
train_x[:3]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 21025,   308,     6,
            3,  1050,   207,     8,  2138,    32,     1,   171,    57,
           15,    49,    81,  5785,    44,   382,   110,   140,    15,
         5194,    60,   154,     9,     1,  4975,  5852,   475,    71,
            5,   260,    12, 21025,   308,    13,  1978,     6,    74,
         2395,     5,   613,    73,     6,  5194,     1, 24103,     5,
         1983, 10166,     1,  5786,  1499,    36,    51,    66,   204,
          145,    67,  1199,  5194, 19869,     1, 37442,     4,     1,
      

In [180]:
def get_batches(x, y, batch_size):
    num_batches = len(x)//batch_size
    x = x[:batch_size * num_batches]
    y=  y[:batch_size * num_batches]
    
    for i in range(0, len(x), batch_size):
        yield x[i : i + batch_size], y[i : i + batch_size]
    

In [196]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.01

## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. Before you run this, make sure the `checkpoints` directory exists.

In [197]:
epoch = 10

with graph.as_default():
    
    saver = tf.train.Saver()

with tf.Session(graph = graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    
    for e in range(epochs):
        
        feed_state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            
            feed = {inputs_ : x, labels_: y[:, None], keep_prob: 1.0, initial_state : feed_state}
            
            loss, feed_state, _ = sess.run([cost, final_state, optimizer], feed_dict = feed)
            
            if iteration % 5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            
            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt") 

Epoch: 0/10 Iteration: 5 Train loss: 0.237
Epoch: 0/10 Iteration: 10 Train loss: 0.241
Epoch: 0/10 Iteration: 15 Train loss: 0.215
Epoch: 0/10 Iteration: 20 Train loss: 0.191
Epoch: 0/10 Iteration: 25 Train loss: 0.186
Val acc: 0.722
Epoch: 0/10 Iteration: 30 Train loss: 0.211
Epoch: 0/10 Iteration: 35 Train loss: 0.219
Epoch: 0/10 Iteration: 40 Train loss: 0.222
Epoch: 1/10 Iteration: 45 Train loss: 0.165
Epoch: 1/10 Iteration: 50 Train loss: 0.206
Val acc: 0.735
Epoch: 1/10 Iteration: 55 Train loss: 0.153
Epoch: 1/10 Iteration: 60 Train loss: 0.131
Epoch: 1/10 Iteration: 65 Train loss: 0.131
Epoch: 1/10 Iteration: 70 Train loss: 0.175
Epoch: 1/10 Iteration: 75 Train loss: 0.119
Val acc: 0.751
Epoch: 1/10 Iteration: 80 Train loss: 0.161
Epoch: 2/10 Iteration: 85 Train loss: 0.147
Epoch: 2/10 Iteration: 90 Train loss: 0.126
Epoch: 2/10 Iteration: 95 Train loss: 0.150
Epoch: 2/10 Iteration: 100 Train loss: 0.148
Val acc: 0.752
Epoch: 2/10 Iteration: 105 Train loss: 0.128
Epoch: 2/10 Ite

## Testing

In [199]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints\sentiment.ckpt
Test accuracy: 0.836
